# Forecast CAC40

In [12]:
import os
import yfinance as yf
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import requests

def get_cac40_tickers():
    """
    Returns the CAC 40 tickers.
    """
    return [
        "AC.PA", "AI.PA", "AIR.PA", "MT.AS", "CS.PA", "BNP.PA", "EN.PA",
        "CAP.PA", "CA.PA", "ACA.PA", "BN.PA", "DSY.PA", "EDEN.PA", "ENGI.PA",
        "EL.PA", "ERF.PA", "RMS.PA", "KER.PA", "OR.PA", "LR.PA", "MC.PA",
        "ML.PA", "ORA.PA", "RI.PA", "PUB.PA", "RNO.PA", "SAF.PA", "SGO.PA",
        "SAN.PA", "SU.PA", "GLE.PA", "STLAP.PA", "STMPA.PA", "TEP.PA",
        "HO.PA", "TTE.PA", "URW.PA", "VIE.PA", "DG.PA", "VIV.PA"
    ]

def get_stock_data(ticker, start_date, end_date):
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if data.empty:
            print(f"{ticker}: No data available.")
            return None

        if isinstance(data.columns, pd.MultiIndex):
            data.columns = [' '.join(col).strip() for col in data.columns.values]

        close_col = next((col for col in data.columns if "Close" in col and ticker in col), None)
        if close_col:
            data.reset_index(inplace=True)
            data = data[['Date', close_col]].rename(columns={'Date': 'ds', close_col: 'y'})
        else:
            print(f"{ticker}: No Close or Adjusted Close column found.")
            return None

        data.dropna(inplace=True)

        if not pd.api.types.is_numeric_dtype(data['y']):
            print(f"{ticker}: Data is not numeric after processing.")
            return None

        if len(data) < 50:
            print(f"{ticker}: Series too short for reliable analysis.")
            return None

        return data
    except Exception as e:
        print(f"{ticker}: Error fetching data - {e}")
        return None

def forecast_stock(data, periods=7):
    try:
        model = Prophet()
        model.fit(data)
        future = model.make_future_dataframe(periods=periods)
        forecast = model.predict(future)
        return forecast, model
    except Exception as e:
        print(f"Error forecasting with Prophet: {e}")
        return None, None

def analyze_tickers(tickers, start_date, end_date, forecast_days=7):
    results = {}
    for ticker in tickers:
        print(f"\nAnalyzing {ticker}...")
        data = get_stock_data(ticker, start_date, end_date)
        if data is not None:
            print(f"{ticker}: Data ready for analysis.")
            forecast, model = forecast_stock(data, periods=forecast_days)
            if forecast is not None:
                print(f"Forecast successful for {ticker}.")
                results[ticker] = {'data': data, 'forecast': forecast, 'model': model}
    return results

def find_top_opportunities(results, forecast_days=7):
    opportunities = []
    for ticker, result in results.items():
        forecast = result['forecast']
        try:
            growth = (forecast['yhat'].iloc[-1] - forecast['yhat'].iloc[-forecast_days]) / forecast['yhat'].iloc[-forecast_days]
            opportunities.append((ticker, growth))
        except Exception as e:
            print(f"{ticker}: Error calculating growth - {e}")
    opportunities = sorted(opportunities, key=lambda x: x[1], reverse=True)
    return opportunities

def save_results_and_images(results, top_opportunities, output_dir="cac40_analysis", top_n=None):
    os.makedirs(output_dir, exist_ok=True)
    ranking_df = pd.DataFrame(top_opportunities, columns=["Ticker", "Growth"])
    ranking_csv_path = os.path.join(output_dir, "ranking.csv")
    ranking_df.to_csv(ranking_csv_path, index=False)
    print(f"\nRanking saved at: {ranking_csv_path}")

    for ticker, _ in top_opportunities[:top_n] if top_n else top_opportunities:
        print(f"Saving predictions for {ticker}...")
        action_dir = os.path.join(output_dir, ticker)
        os.makedirs(action_dir, exist_ok=True)

        forecast = results[ticker]['forecast']
        model = results[ticker]['model']
        fig = model.plot(forecast)
        fig_path = os.path.join(action_dir, f"{ticker}_forecast.png")
        fig.savefig(fig_path)
        plt.close(fig)

if __name__ == "__main__":
    start_date = '2020-01-01'
    end_date = '2024-11-28'
    forecast_days = 2

    print("Fetching CAC 40 tickers...")
    cac40_tickers = get_cac40_tickers()
    print(f"{len(cac40_tickers)} tickers retrieved.")

    print("\nAnalyzing stocks...")
    results = analyze_tickers(cac40_tickers, start_date, end_date, forecast_days)

    print("\nIdentifying top investment opportunities...")
    top_opportunities = find_top_opportunities(results, forecast_days=forecast_days)
    print("\nTop investment opportunities (predicted growth):")
    for ticker, growth in top_opportunities[:10]:
        print(f"{ticker}: Predicted growth: {growth:.2%}")

    print("\nSaving results...")
    save_results_and_images(results, top_opportunities, output_dir="cac40_analysis", top_n=10)


Fetching CAC 40 tickers...
40 tickers retrieved.

Analyzing stocks...

Analyzing AC.PA...
AC.PA: Data ready for analysis.


10:34:41 - cmdstanpy - INFO - Chain [1] start processing
10:34:41 - cmdstanpy - INFO - Chain [1] done processing


Forecast successful for AC.PA.

Analyzing AI.PA...
AI.PA: Data ready for analysis.


10:34:42 - cmdstanpy - INFO - Chain [1] start processing
10:34:42 - cmdstanpy - INFO - Chain [1] done processing


Forecast successful for AI.PA.

Analyzing AIR.PA...
AIR.PA: Data ready for analysis.


...

Forecast successful for VIE.PA.

Analyzing DG.PA...
DG.PA: Data ready for analysis.


10:35:10 - cmdstanpy - INFO - Chain [1] start processing
10:35:10 - cmdstanpy - INFO - Chain [1] done processing


Forecast successful for DG.PA.

Analyzing VIV.PA...
VIV.PA: Data ready for analysis.


10:35:11 - cmdstanpy - INFO - Chain [1] start processing
10:35:11 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\mania\anaconda3\Lib\site-packages\prophet\plot.py:72: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  fcst_t = fcst['ds'].dt.to_pydatetime()
C:\Users\mania\anaconda3\Lib\site-packages\prophet\plot.py:73: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  ax.plot(m.history['ds'].dt.to_pydatetime(), m.history['y'], 'k.',


Forecast successful for VIV.PA.

Identifying top investment opportunities...

Top investment opportunities (predicted growth):
STMPA.PA: Predicted growth: 0.51%
ERF.PA: Predicted growth: 0.47%
AIR.PA: Predicted growth: 0.43%
AC.PA: Predicted growth: 0.37%
DSY.PA: Predicted growth: 0.36%
SAF.PA: Predicted growth: 0.35%
SU.PA: Predicted growth: 0.32%
CAP.PA: Predicted growth: 0.31%
BNP.PA: Predicted growth: 0.22%
AI.PA: Predicted growth: 0.21%

Saving results...

Ranking saved at: cac40_analysis\ranking.csv
Saving predictions for STMPA.PA...
Saving predictions for ERF.PA...
Saving predictions for AIR.PA...
Saving predictions for AC.PA...
Saving predictions for DSY.PA...
Saving predictions for SAF.PA...
Saving predictions for SU.PA...
Saving predictions for CAP.PA...
Saving predictions for BNP.PA...
Saving predictions for AI.PA...


# Forecast S&P500

In [26]:
import os
import yfinance as yf
from prophet import Prophet
import pandas as pd
import matplotlib.pyplot as plt
import requests
import numpy as np
import warnings

# Étape 1 : Récupérer les tickers du S&P 500
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    tables = pd.read_html(response.text)
    sp500_table = tables[0]  # La première table contient la liste des actions
    tickers = sp500_table['Symbol'].tolist()
    return tickers

# Étape 2 : Récupérer et nettoyer les données d'une action
def get_stock_data(ticker, start_date, end_date):
    try:
        # Télécharger les données
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if data.empty:
            print(f"{ticker}: Pas de données disponibles.")
            return None

        # Vérifier et aplatir les colonnes multi-index
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = [' '.join(col).strip() for col in data.columns.values]

        # Identifier la colonne `Close` ou `Adj Close` contenant le ticker
        close_col = next((col for col in data.columns if "Close" in col and ticker in col), None)
        if close_col:
            data.reset_index(inplace=True)
            data = data[['Date', close_col]].rename(columns={'Date': 'ds', close_col: 'y'})
        else:
            print(f"{ticker}: Aucune colonne Close ou Adj Close trouvée.")
            return None

        # Supprimer les valeurs manquantes
        data.dropna(inplace=True)

        # Vérifier si les données sont numériques
        if not pd.api.types.is_numeric_dtype(data['y']):
            print(f"{ticker}: Les données ne sont pas numériques après traitement.")
            return None

        # Vérifier la longueur minimale
        if len(data) < 50:
            print(f"{ticker}: Série trop courte pour une analyse fiable.")
            return None

        return data
    except Exception as e:
        print(f"{ticker}: Erreur lors de la récupération des données - {e}")
        return None

# Étape 3 : Effectuer une prévision avec Prophet
def forecast_stock(data, periods=9):
    try:
        model = Prophet()
        model.fit(data)
        future = model.make_future_dataframe(periods=periods)
        forecast = model.predict(future)
        return forecast, model
    except Exception as e:
        print(f"Erreur lors de la prévision avec Prophet: {e}")
        return None, None

# Étape 4 : Analyser les tickers du S&P 500
def analyze_sp500_tickers(tickers, start_date, end_date, forecast_days=9):
    results = {}
    for ticker in tickers:
        print(f"\nAnalyse de {ticker}...")
        data = get_stock_data(ticker, start_date, end_date)
        if data is not None:
            print(f"{ticker}: Données prêtes pour l'analyse.")
            forecast, model = forecast_stock(data, periods=forecast_days)
            if forecast is not None:
                print(f"Prévision réussie pour {ticker}.")
                results[ticker] = {'data': data, 'forecast': forecast, 'model': model}
    return results

# Étape 5 : Trouver les meilleures opportunités d'investissement
def find_top_opportunities(results, forecast_days=9):
    opportunities = []
    for ticker, result in results.items():
        forecast = result['forecast']
        try:
            # Calculer le taux de croissance prévu
            growth = (forecast['yhat'].iloc[-1] - forecast['yhat'].iloc[-forecast_days]) / forecast['yhat'].iloc[-forecast_days]
            opportunities.append((ticker, growth))
        except Exception as e:
            print(f"{ticker}: Erreur lors du calcul de la croissance prévue - {e}")
    # Trier par taux de croissance décroissant
    opportunities = sorted(opportunities, key=lambda x: x[1], reverse=True)
    return opportunities

# Étape 6 : Générer des graphiques et un CSV
def save_results_and_images(results, top_opportunities, output_dir="results", top_n=None):
    # Créer le répertoire principal
    os.makedirs(output_dir, exist_ok=True)

    # Sauvegarder les classements dans un CSV
    ranking_df = pd.DataFrame(top_opportunities, columns=["Ticker", "Growth"])
    ranking_csv_path = os.path.join(output_dir, "ranking.csv")
    ranking_df.to_csv(ranking_csv_path, index=False)
    print(f"\nClassement sauvegardé dans : {ranking_csv_path}")

    # Sauvegarder les graphiques dans des dossiers par action
    for ticker, _ in top_opportunities[:top_n] if top_n else top_opportunities:
        print(f"Enregistrement des prévisions pour {ticker}...")
        action_dir = os.path.join(output_dir, ticker)
        os.makedirs(action_dir, exist_ok=True)

        forecast = results[ticker]['forecast']
        model = results[ticker]['model']

        # Sauvegarder le graphique des prévisions
        fig = model.plot(forecast)
        fig_path = os.path.join(action_dir, f"{ticker}_forecast.png")
        fig.savefig(fig_path)
        plt.close(fig)  # Fermer le graphique pour économiser de la mémoire

# Étape 7 : Exécution principale
if __name__ == "__main__":
    # Paramètres
    start_date = '2020-01-01'
    end_date = '2024-11-28'
    forecast_days = 9

    # Récupérer les tickers du S&P 500
    print("Récupération des tickers du S&P 500...")
    sp500_tickers = get_sp500_tickers()
    print(f"{len(sp500_tickers)} tickers récupérés.")

    # Analyser les tickers
    print("\nAnalyse des actions...")
    results = analyze_sp500_tickers(sp500_tickers, start_date, end_date, forecast_days)

    # Identifier les meilleures opportunités
    print("\nIdentification des meilleures opportunités...")
    top_opportunities = find_top_opportunities(results, forecast_days=forecast_days)
    print("\nTop opportunités d'investissement (croissance prévue) :")
    for ticker, growth in top_opportunities[:10]:  # Top 10 opportunités
        print(f"{ticker}: Croissance prévue: {growth:.2%}")

    # Sauvegarder les résultats et les images
    print("\nSauvegarde des résultats...")
    save_results_and_images(results, top_opportunities, output_dir="9daysp500_analysis", top_n=10)


Récupération des tickers du S&P 500...


C:\Users\mania\AppData\Local\Temp\ipykernel_10564\908370912.py:14: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(response.text)


503 tickers récupérés.

Analyse des actions...

Analyse de MMM...
MMM: Données prêtes pour l'analyse.


10:58:23 - cmdstanpy - INFO - Chain [1] start processing
10:58:24 - cmdstanpy - INFO - Chain [1] done processing


Prévision réussie pour MMM.

Analyse de AOS...
AOS: Données prêtes pour l'analyse.


...

Prévision réussie pour ZBH.

Analyse de ZTS...
ZTS: Données prêtes pour l'analyse.


11:06:28 - cmdstanpy - INFO - Chain [1] start processing
11:06:29 - cmdstanpy - INFO - Chain [1] done processing


Prévision réussie pour ZTS.

Identification des meilleures opportunités...

Top opportunités d'investissement (croissance prévue) :
ALGN: Croissance prévue: 8.93%
ENPH: Croissance prévue: 8.81%
TER: Croissance prévue: 7.77%
QCOM: Croissance prévue: 7.49%
AMD: Croissance prévue: 7.46%
PODD: Croissance prévue: 6.63%
ALB: Croissance prévue: 6.41%
ZBRA: Croissance prévue: 6.33%
EXPE: Croissance prévue: 6.03%
FFIV: Croissance prévue: 6.02%

Sauvegarde des résultats...

Classement sauvegardé dans : 9daysp500_analysis\ranking.csv
Enregistrement des prévisions pour ALGN...


C:\Users\mania\anaconda3\Lib\site-packages\prophet\plot.py:72: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  fcst_t = fcst['ds'].dt.to_pydatetime()
C:\Users\mania\anaconda3\Lib\site-packages\prophet\plot.py:73: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  ax.plot(m.history['ds'].dt.to_pydatetime(), m.history['y'], 'k.',


Enregistrement des prévisions pour ENPH...
Enregistrement des prévisions pour TER...
Enregistrement des prévisions pour QCOM...
Enregistrement des prévisions pour AMD...
Enregistrement des prévisions pour PODD...
Enregistrement des prévisions pour ALB...
Enregistrement des prévisions pour ZBRA...
Enregistrement des prévisions pour EXPE...
Enregistrement des prévisions pour FFIV...
